In [1]:
from copy import deepcopy
import torch

from yolov7_utils.torch_utils import is_parallel
from nas.supernet.yolo import YOLOModel
from nas.supernet.supernet_yolov7 import YOLOSuperNet

In [2]:
supernet = YOLOSuperNet(cfg='./yaml/yolov7_supernet.yml', ch=3)
net = YOLOModel(cfg='./yaml/yolov7.yaml', ch=3)

c:\Users\ai-ws\anaconda3\envs\torch\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
# It needs models module in yolov7 to load weights when it was saved.
weights = 'yolov7.pt'
device = 'cpu'
ckpt = torch.load(weights, map_location=device)

In [8]:
for superkey, yolokey in zip(supernet.state_dict(), ckpt['model'].state_dict()):
    print(f'{superkey} | {yolokey}')

model.0.conv.weight | model.0.conv.weight
model.0.bn.weight | model.0.bn.weight
model.0.bn.bias | model.0.bn.bias
model.0.bn.running_mean | model.0.bn.running_mean
model.0.bn.running_var | model.0.bn.running_var
model.0.bn.num_batches_tracked | model.0.bn.num_batches_tracked
model.1.conv.weight | model.1.conv.weight
model.1.bn.weight | model.1.bn.weight
model.1.bn.bias | model.1.bn.bias
model.1.bn.running_mean | model.1.bn.running_mean
model.1.bn.running_var | model.1.bn.running_var
model.1.bn.num_batches_tracked | model.1.bn.num_batches_tracked
model.2.conv.weight | model.2.conv.weight
model.2.bn.weight | model.2.bn.weight
model.2.bn.bias | model.2.bn.bias
model.2.bn.running_mean | model.2.bn.running_mean
model.2.bn.running_var | model.2.bn.running_var
model.2.bn.num_batches_tracked | model.2.bn.num_batches_tracked
model.3.conv.weight | model.3.conv.weight
model.3.bn.weight | model.3.bn.weight
model.3.bn.bias | model.3.bn.bias
model.3.bn.running_mean | model.3.bn.running_mean
model.3.

In [9]:
for key in supernet.state_dict():
    print(key)

model.0.conv.weight
model.0.bn.weight
model.0.bn.bias
model.0.bn.running_mean
model.0.bn.running_var
model.0.bn.num_batches_tracked
model.1.conv.weight
model.1.bn.weight
model.1.bn.bias
model.1.bn.running_mean
model.1.bn.running_var
model.1.bn.num_batches_tracked
model.2.conv.weight
model.2.bn.weight
model.2.bn.bias
model.2.bn.running_mean
model.2.bn.running_var
model.2.bn.num_batches_tracked
model.3.conv.weight
model.3.bn.weight
model.3.bn.bias
model.3.bn.running_mean
model.3.bn.running_var
model.3.bn.num_batches_tracked
model.4.layers.0.conv.weight
model.4.layers.0.bn.weight
model.4.layers.0.bn.bias
model.4.layers.0.bn.running_mean
model.4.layers.0.bn.running_var
model.4.layers.0.bn.num_batches_tracked
model.4.layers.1.conv.weight
model.4.layers.1.bn.weight
model.4.layers.1.bn.bias
model.4.layers.1.bn.running_mean
model.4.layers.1.bn.running_var
model.4.layers.1.bn.num_batches_tracked
model.4.layers.2.conv.weight
model.4.layers.2.bn.weight
model.4.layers.2.bn.bias
model.4.layers.2.bn

In [10]:
# if we use IDetect layer, it has 6 more state_dict than yolov7.pt
# because IDetect layer has 6 more parameters than Detect layer.
len(supernet.state_dict()), len(ckpt['model'].state_dict())

(560, 560)

In [11]:
def load_weights_from_yolo(supernet, yolo_dict):
        super_dict = supernet.state_dict()

        for superkey, yolokey in zip(super_dict, yolo_dict):
            super_dict[superkey] = yolo_dict[yolokey] # yolo state와 supernet state 1:1 매칭

        supernet.load_state_dict(super_dict) # load weight
        print('All keys matched successfully')
        
        return supernet

In [12]:
supernet_weighted = load_weights_from_yolo(supernet, ckpt['model'].state_dict())

All keys matched successfully


In [62]:
ckpt

YOLOSuperNet(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (3): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU()
    )
    (4): BBoneELAN(
      (layers): Sequential(
        (0

In [13]:
ckpt = {
        'model': deepcopy(supernet_weighted.module if is_parallel(supernet_weighted) else supernet_weighted).half(),
}

torch.save(ckpt, 'yolov7_supernet.pt')

In [14]:
for m in supernet_weighted.model:
    print(m)
    break

Conv(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU()
)


In [87]:
torch.save(torch.load('yolov7.pt')['model'].state_dict(), 'yolov7_statedict.pt') # save yolov7 statedict
yolo_weight = torch.load('yolov7_statedict.pt') # load yolov7 statedict

In [88]:
supernet_pt = load_weights_from_yolo(supernet, yolo_weight)

All keys matched successfully


In [89]:
# check weight
x = supernet_weighted.state_dict()['model.4.layers.1.conv.weight'].cuda()
y = yolo_weight['model.5.conv.weight']

print('Supernet BBoneELAN Conv // shape :', x.shape)
print('weight :', x.flatten())
print('---')
print('YOLOv7 BBoneELAN Conv  // shape :', y.shape)
print('weight :', y.flatten())
print('---')
print('Is same?: ', (x == y).flatten())

Supernet BBoneELAN Conv // shape : torch.Size([64, 128, 1, 1])
weight : tensor([-0.10382,  0.02238, -0.04446,  ..., -0.05933,  0.02414,  0.00448], device='cuda:0')
---
YOLOv7 BBoneELAN Conv  // shape : torch.Size([64, 128, 1, 1])
weight : tensor([-0.10382,  0.02238, -0.04446,  ..., -0.05933,  0.02414,  0.00448], device='cuda:0', dtype=torch.float16)
---
Is same?:  tensor([True, True, True,  ..., True, True, True], device='cuda:0')


In [91]:
torch.save({'model': supernet_pt}, 'yolov7_supernet_pt.pt') # save supernet statedict